In [1]:
import pandas as pd

from scipy.stats import iqr

import wrangle_zillow


In [2]:
df = wrangle_zillow.get_zillow_data()

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
def find_outlier_columns(df):
    """function takes in dataframe and returns list of column names:
    columns that are floats
    columns that are not categorical data values
    """
    outlier_columns = [x for x in df.columns if df[x].dtype == 'float64']
    outlier_columns.remove('longitude')
    outlier_columns.remove('latitude')
    for x in outlier_columns:
        y = len(df[x].value_counts())
        if y < 6:
            outlier_columns.remove(x)
        elif x.find('id') > 0:
            outlier_columns.remove(x)
    return outlier_columns

In [4]:
outlier_columns = find_outlier_columns(df)

In [5]:
df.columns

Index(['id', 'parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardb

In [6]:
outlier_columns = ['bathroomcnt', 'bedroomcnt', 'calculatedbathnbr', 'taxvaluedollarcnt']

In [7]:
def get_thresholds(df=df, columns=outlier_columns, k=3):
    list_upper = []
    list_lower = []
    for col in outlier_columns:
        lower_25 = (df[col].quantile(.25))
        upper_75 = (df[col].quantile(.75))
        IQR = iqr(df[col])*k
        upper_threshold = upper_75 + IQR
        if df[col].min() >= 0:
            lower_threshold = max([lower_25 - IQR, 0])
        else:
            lower_threshold = lower_25 - IQR
        list_upper.append(upper_threshold)
        list_lower.append(lower_threshold)
    return list_upper, list_lower

In [8]:
list_upper, list_lower = get_thresholds(df=df, columns=outlier_columns, k=3)

In [9]:
def replace_outliers(df=df, outliers_columns=outlier_columns, list_upper=list_upper, list_lower=list_lower):
    for i in range(0, len(outlier_columns)):
        col = outlier_columns[i]
        rebuild_col = []
        for x in df[col]:
            if x > list_upper[i]:
                rebuild_col.append(list_upper[i])
            elif x < list_lower[i]:
                rebuild_col.append(list_upper[i])
            else:
                rebuild_col.append(x)
        df[col] = rebuild_col
    return df  

In [10]:
df = replace_outliers(df=df, outliers_columns=outlier_columns, list_upper=list_upper, list_lower=list_lower)

In [11]:
df.taxvaluedollarcnt.max()

49061236.0

In [12]:
df.taxvaluedollarcnt.value_counts()

400000.0    56
600000.0    50
350000.0    49
450000.0    45
455000.0    44
            ..
247786.0     1
820528.0     1
302891.0     1
92106.0      1
98304.0      1
Name: taxvaluedollarcnt, Length: 50949, dtype: int64

In [ ]:
outlier_columns